In [ ]:
!pip install kagglehub lime

In [7]:
# import
import kagglehub
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

from lime import lime_image
from skimage.segmentation import mark_boundaries

In [8]:
%load_ext tensorboard
%tensorboard --logdir runs

In [9]:
# Constants
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SUBSET_SIZE = 2000
BATCH_SIZE = 32
EPOCHS = 100
LEARNING_RATE = 1e-4
SEED=42
NUM_WORKERS=2

In [10]:
np.random.seed(SEED)
torch.manual_seed(SEED)

In [11]:
# Download dataset
path = kagglehub.dataset_download("kmader/skin-cancer-mnist-ham10000")
print("Dataset:", path)

Dataset: /home/lois/.cache/kagglehub/datasets/kmader/skin-cancer-mnist-ham10000/versions/2


In [12]:
# Dataset loading
image_paths_dict = {}
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.jpg'):
            image_id = os.path.splitext(file)[0]
            image_paths_dict[image_id] = os.path.join(root, file)

metadata_path = os.path.join(path, 'HAM10000_metadata.csv')

df = pd.read_csv(metadata_path)
df['path'] = df['image_id'].map(image_paths_dict.get)
df = df.dropna(subset=['path'])

lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}
df['cell_type'] = df['dx'].map(lesion_type_dict.get)

# Encodage des labels (texte -> 0, 1, 2...)
le = LabelEncoder()
df['label'] = le.fit_transform(df['cell_type'])
NUM_CLASSES = len(le.classes_)


In [13]:
# Creation subset / dataset
if SUBSET_SIZE < len(df):
    df_subset = df.sample(n=SUBSET_SIZE, random_state=SEED).reset_index(drop=True)
else:
    df_subset = df

train_df, test_df = train_test_split(df_subset, test_size=0.2, stratify=df_subset['label'], random_state=SEED)
# Définition des transformations (Prétraitement pour ResNet)
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(), # Augmentation de données simple
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Classe Dataset PyTorch personnalisée
class SkinDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['path']
        image = Image.open(img_path).convert('RGB')
        label = torch.tensor(int(self.df.iloc[idx]['label']), dtype=torch.long)
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

train_dataset = SkinDataset(train_df, transform=data_transforms['train'])
test_dataset = SkinDataset(test_df, transform=data_transforms['val'])
train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True
)
test_loader = DataLoader(
    test_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

In [14]:
# Transfer Learning avec ResNet18
model = models.resnet18(pretrained=True)

for param in model.parameters(): # freeze all layers
    param.requires_grad = False

# Replace last layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, NUM_CLASSES)
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=LEARNING_RATE)


/home/lois/Documents/mlbio/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/lois/Documents/mlbio/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [16]:
# Training
writer = SummaryWriter('runs/skin_cancer_experiment')
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        writer.add_scalar('Training/Batch_Loss', loss.item(), epoch * len(train_loader) + i)
    
    epoch_acc = 100 * correct / total
    epoch_loss = running_loss / len(train_loader) 
    
    # Validation
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
            
    val_acc = 100 * correct_val / total_val
    val_loss /= len(test_loader)
    
    writer.add_scalars('Loss', {'Train': epoch_loss, 'Val': val_loss}, epoch)
    writer.add_scalars('Accuracy', {'Train': epoch_acc, 'Val': val_acc}, epoch)

    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.2f}% | Val Acc: {val_acc:.2f}%")
    
writer.close()

/home/lois/Documents/mlbio/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 1/100 | Loss: 1.3794 | Acc: 56.50% | Val Acc: 66.75%
Epoch 2/100 | Loss: 1.1248 | Acc: 66.69% | Val Acc: 66.75%
Epoch 3/100 | Loss: 1.0623 | Acc: 66.88% | Val Acc: 66.75%
Epoch 4/100 | Loss: 1.0280 | Acc: 67.12% | Val Acc: 66.75%
Epoch 5/100 | Loss: 1.0020 | Acc: 67.56% | Val Acc: 66.50%


KeyboardInterrupt: 

In [ ]:
# Save model
torch.save(model.state_dict(), 'skin_lesion_resnet18.pth')

In [ ]:
# ==========================================
# 6. INTERPRÉTABILITÉ AVEC LIME
# ==========================================
print("\n--- Démarrage de LIME ---")

# Fonction de prédiction spéciale pour LIME
# LIME passe une image numpy (H, W, 3), il faut la convertir en Tensor PyTorch
def batch_predict(images):
    model.eval()
    batch = torch.stack([data_transforms['val'](Image.fromarray(i)) for i in images], dim=0)
    batch = batch.to(DEVICE)
    
    with torch.no_grad():
        logits = model(batch)
        probs = F.softmax(logits, dim=1)
        
    return probs.detach().cpu().numpy()

# Initialisation de l'explainer
explainer = lime_image.LimeImageExplainer()

# Prendre une image au hasard du set de test pour l'expliquer
idx = np.random.randint(0, len(test_dataset))
img_tensor, label_idx = test_dataset[idx]

# Convertir le tenseur en image numpy pour LIME (denormalization)
# Note: L'image passée à LIME doit ressembler à une photo normale
img_numpy = np.array(Image.open(test_df.iloc[idx]['path']).convert('RGB').resize((224, 224)))

print(f"Explication de l'image {idx}. Vraie classe : {le.classes_[label_idx]}")

# Génération de l'explication
explanation = explainer.explain_instance(
    img_numpy, 
    batch_predict, # Notre fonction de prédiction
    top_labels=5, 
    hide_color=0, 
    num_samples=1000 # Nombre de perturbations
)

# Visualisation
temp, mask = explanation.get_image_and_mask(
    explanation.top_labels[0], 
    positive_only=True, 
    num_features=5, 
    hide_rest=False
)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(img_numpy)
plt.title("Image Originale")
plt.axis('off')

plt.subplot(1, 2, 2)
# mark_boundaries dessine les contours des superpixels importants
plt.imshow(mark_boundaries(temp / 255 + 0.5, mask)) # Petit ajustement de luminosité pour l'affichage
plt.title(f"Explication LIME\n(Prédiction: {le.classes_[explanation.top_labels[0]]})")
plt.axis('off')
plt.tight_layout()
plt.show()